In [ ]:
%pylab inline

# JSAnimation import available at https://github.com/jakevdp/JSAnimation
from JSAnimation.JSAnimation import IPython_display
from matplotlib import animation

import numpy as np

from intern.remote.boss import BossRemote
from intern.resource.boss.resource import ChannelResource


## Setup `intern` Remote Instance

Create an instance of the Boss Python SDK intern.  You need to provide configuration details to the library.

- Copy `./boss.cfg.example` to `./boss.cfg`
- Get your API Token: https://api.theboss.io/token
- Copy and paste your API token into `./boss.cfg`

In [ ]:
rmt = BossRemote("./boss.cfg")

## Create a Channel Resource

We interact with the Boss via "resources" which represent how data is organized in the Boss.  `Channels` store image data.  Let's create

In [ ]:
# Create a resource for the channel you want to access. Let's use a publically available dataset
collection_name = "my_col"
experiment_name = "my_exp"
channel_name = "my_ch"
channel = ChannelResource(channel_name, collection_name, experiment_name)
channel = rmt.get_project(channel)

## Set Cutout Arguments
Set the spatial extent for the cutout.  Here we are doing a cutout of non-time series data, but if the channel you were accessing contained multiple time samples, you could simply add a time range to the query.  See the `intern` docs for more detail:

- Wiki: https://github.com/jhuapl-boss/intern/wiki
- Docs: https://jhuapl-boss.github.io/intern/
- Examples: https://github.com/jhuapl-boss/intern/tree/master/examples

In [ ]:
x_rng = [0, 512]
y_rng = [0, 512]
z_slice = 100
t_rng = [0, 10]
time_data1 = rmt.get_cutout(channel, 0, x_rng, y_rng, [z_slice,z_slice+1], t_rng)
time_data1.shape

In [ ]:
apply_normalization = True
apply_hist_eq = False
time_data = time_data1
np.max(time_data)

In [ ]:
hist_data = np.histogram(time_data, bins=256)
min_val = hist_data[1][0]
max_val = hist_data[1][-1]
print("Min: {}".format(min_val))
print("Max: {}".format(max_val))

In [ ]:
def normalize(image_in):
    val_min = float(image_in.min())
    val_max = float(image_in.max())
    
    data_equalized = np.zeros(image_in.shape)
    for i in range(image_in.shape[0]):
        data_equalized[i, 0, :, :] = np.floor((image_in[i, 0, :, :]-val_min)/(val_max-val_min)*255.)
        
    return data_equalized

def histogram_eq_uint8(image_in, number_bins=256):
    # from http://www.janeriksolem.net/2009/06/histogram-equalization-with-python-and.html

    # get image histogram
    image_histogram, bins = np.histogram(image_in.flatten(), number_bins, normed=True)
    cdf = image_histogram.cumsum() # cumulative distribution function
    cdf = 255 * cdf / cdf[-1] # normalize

    data_equalized = np.zeros(image_in.shape)
    for i in range(image_in.shape[0]):
        # use linear interpolation of cdf to find new pixel values
        image_equalized = np.interp(image_in[i, 0, :, :].flatten(), bins[:-1], cdf)
        data_equalized[i, 0, :, :] = image_equalized.reshape(image_in[i, 0, :, :].shape)
    return data_equalized

if apply_hist_eq:
    time_data = histogram_eq_uint8(time_data)
    
if apply_normalization:
    time_data = normalize(time_data)

In [ ]:
time_data = np.array(time_data, dtype=np.uint8)

In [ ]:
print(time_data.min())
print(time_data.max())
print(time_data.dtype)
print(time_data.shape)

In [ ]:
fig = plt.figure()
im = plt.imshow(time_data[0,0,:,:], cmap=plt.cm.gray, interpolation='bicubic') 

def init():
    im.set_data(time_data[0,0,:,:])

def animate(i): 
    im.set_data(time_data[i,0,:,:])
    return im

animation.FuncAnimation(fig, animate, init_func=init, frames=time_data.shape[0],
                               interval=75)